In [2]:
import random
import numpy as np
import pandas as pd
import math
import openpyxl
import time
import sys
np.set_printoptions(threshold=sys.maxsize)
import xlwings
filename1 = "C:\\Users\\admin\\PYTHON\\Thesis_Meta\\GA data.xlsx"
wb1=xlwings.Book(filename1)
sheet1=wb1.sheets[0]

#paramters
period=int(sheet1.range("D3").value)
supermarket=int(sheet1.range("D4").value)
item = int(sheet1.range("D5").value)
pc=sheet1.range("D6:F6").value
hc=sheet1.range("D12:F12").value
oc=int(sheet1.range("D15").value)
WU=int(sheet1.range("D16").value)
m = 100  #number of chromosomes - population size
n = (3, 3, 12)
shape = (3,3,12)
# Parameter of GA
pselect = 0.15
pmutation = 0.35
maxI_ga = 15
# Starting time
start_time = time.time()
a=sheet1.range("A31:A138").value
b=sheet1.range("B31:B138").value
c=sheet1.range("C31:C138").value
initial_inventory_data = sheet1.range('K2:M10').value 
a = np.array(a)
b = np.array(b)
c = np.array(c)
calculated_values=((a + 2 * b + c)/4)
Demand=np.ceil(calculated_values.reshape((3,3,12)))

def initialize(m, n, p):
    v = np.zeros((m, n, p), dtype=int)
    for i in range(m):
        for j in range(n):
            for k in range(p):
                v[i][j][k] = np.random.randint(0, 2, dtype=int)
    return v
v_list = [initialize(*shape) for _ in range(m)]

In [13]:
I = np.zeros_like(Demand)  # Inventory levels
initial_inventory = {}
for row in initial_inventory_data:
    if row[0] is not None and row[1] is not None and row[2] is not None:  # Ensure all values are present
        i, j = int(row[0]), int(row[1])
        inventory = row[2]
        initial_inventory[(i, j)] = inventory
for (i, j), inv in initial_inventory.items():
    I[i, j, 0] = inv
print(I)

[[[1200.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
      0.]
  [1000.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
      0.]
  [ 600.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
      0.]]

 [[1800.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
      0.]
  [1000.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
      0.]
  [   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
      0.]]

 [[ 200.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
      0.]
  [ 500.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
      0.]
  [1200.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
      0.]]]


In [14]:
def check_chromosome(v_list, Demand, WU, initial_inventory):
    modified_v_list = []
    for v in v_list:
        v_copy = np.copy(v)
        cumulative_Demand = np.zeros_like(Demand)

        for i in range(v_copy.shape[0]):
            for j in range(v_copy.shape[1]):
                cumulative = 0
                for k in range(v_copy.shape[2]):
                    # Check if initial inventory can cover cumulative demand up to period k
                    if I[i, j, 0] >= np.sum(Demand[i, j, :k+1]):
                        v_copy[i, j, k] = 0
                        if k+1 < v_copy.shape[2]:
                            v_copy[i, j, k+1] = 1
                        cumulative_Demand[i, j, k] = 0 
                        cumulative_Demand[i, j, k+1] = Demand[i, j, k+1]
                    if I[i, j, 0]<Demand[i, j, 0]:
                        v_copy[i, j, 0] = 1
                        cumulative_Demand[i, j, 0] = Demand[i, j, 0]
                    # Adjust based on WU and WL
                    if v_copy[i, j, k] == 1:
                        temp1 = k
                        temp2 = k
                        cumulative_Demand[i, j, temp1] = Demand[i, j, temp1]
                        
                        # Look ahead to find consecutive zeros in v_copy
                        if (k+1) < v_copy.shape[2] and v_copy[i, j, k+1] == 0:
                            while (temp2 + 1) < v_copy.shape[2] and v_copy[i, j, temp2 + 1] == 0:
                                temp2 += 1
                            cumulative_Demand[i, j, temp1] = np.sum(Demand[i, j, temp1:temp2 + 1])

                    if cumulative_Demand[i, j, k] > WU:
                        v_copy[i, j, k] = 1
                        cumulative_Demand[i, j, k] = Demand[i, j, k]
                       
        modified_v_list.append(v_copy)
    return modified_v_list
modified_v_list = check_chromosome(v_list, Demand, WU, initial_inventory)
for i, modified_v in enumerate(modified_v_list, start=1):
    print(f"Modified v[{i}]:\n{modified_v}\n")


Modified v[1]:
[[[0 1 1 1 1 1 0 0 1 0 1 0]
  [0 0 1 1 1 0 0 0 0 0 0 1]
  [1 1 1 0 1 0 0 1 0 0 0 1]]

 [[0 1 1 0 0 0 1 1 0 1 1 1]
  [0 1 1 1 1 0 0 1 0 1 0 0]
  [1 0 0 1 1 1 0 1 1 1 0 1]]

 [[1 1 1 0 1 0 0 0 1 1 0 1]
  [1 0 0 0 1 1 1 1 0 1 1 0]
  [0 0 1 0 0 1 0 0 1 0 1 0]]]

Modified v[2]:
[[[0 1 0 0 1 0 1 1 0 1 1 0]
  [0 0 1 0 0 0 0 1 0 0 0 1]
  [1 1 0 0 0 0 0 0 0 0 0 0]]

 [[0 1 1 0 0 1 0 1 1 0 1 1]
  [0 1 0 0 1 1 0 0 1 1 0 1]
  [1 1 1 1 1 0 0 1 1 1 1 0]]

 [[1 0 0 0 0 0 0 1 0 0 1 1]
  [1 0 0 1 0 0 1 0 1 1 1 0]
  [0 0 1 1 1 0 1 0 0 1 1 0]]]

Modified v[3]:
[[[0 1 0 1 0 1 1 0 0 0 0 0]
  [0 0 1 0 1 0 1 0 0 0 1 0]
  [1 1 1 0 1 0 0 1 0 1 1 1]]

 [[0 1 1 1 1 0 0 1 0 1 0 0]
  [0 1 0 1 1 1 0 0 1 0 0 1]
  [1 0 1 1 1 1 1 0 1 1 1 1]]

 [[1 0 1 0 1 1 1 1 1 1 0 0]
  [1 1 0 0 0 0 0 0 1 1 1 1]
  [0 0 1 0 1 1 1 0 0 1 0 0]]]

Modified v[4]:
[[[0 1 0 1 0 0 1 1 0 0 1 0]
  [0 0 1 0 1 0 0 0 1 0 0 1]
  [1 1 1 1 1 1 1 0 1 0 0 1]]

 [[0 1 0 1 1 0 1 0 1 0 1 1]
  [0 1 0 1 0 0 0 1 0 1 1 0]
  [1 1 0 1 1 1 1 1 1 

In [15]:
def cumuD(modified_v_list, Demand):
    cumulative_Demand_list = []
    for v in modified_v_list:
        cumul_Demand = np.zeros_like(Demand)
        for i in range(v.shape[0]):
            for j in range(v.shape[1]):
                cumulative = 0
                for k in range(v.shape[2]):
                    # Adjust based on WU and WL
                    if v[i, j, k] == 1:
                        temp1 = k
                        temp2 = k
                        cumul_Demand[i, j, temp1] = Demand[i, j, temp1]
                        # Look ahead to find consecutive zeros in v_copy
                        if (k + 1) < v.shape[2] and v[i, j, k + 1] == 0:
                            while (temp2 + 1) < v.shape[2] and v[i, j, temp2 + 1] == 0:
                                temp2 += 1
                            cumul_Demand[i, j, temp1] = np.sum(Demand[i, j, temp1:temp2 + 1])
        cumulative_Demand_list.append(cumul_Demand)
    return cumulative_Demand_list
cumulative_Demand_list = cumuD(modified_v_list, Demand)
for i, cumulative_Demand in enumerate(cumulative_Demand_list, start=1):
    print(f"Cumulative Demand[{i}]:\n{cumulative_Demand}\n")

Cumulative Demand[1]:
[[[   0. 1144. 1301. 1328. 1092. 3348.    0.    0. 2184.    0. 2169.
      0.]
  [   0.    0.  470.  429. 3781.    0.    0.    0.    0.    0.    0.
    445.]
  [1010. 1130. 2155.    0. 3082.    0.    0. 4237.    0.    0.    0.
   1033.]]

 [[   0. 1204. 3958.    0.    0.    0. 1106. 1967.    0. 1029.  948.
    966.]
  [   0.  664.  735.  754. 2092.    0.    0. 1391.    0. 1931.    0.
      0.]
  [1537.    0.    0.  520.  501.  931.    0.  535.  562. 1071.    0.
    539.]]

 [[ 243.  330.  532.    0. 1208.    0.    0.    0.  254.  675.    0.
    266.]
  [4812.    0.    0.    0. 1227. 1063. 1185. 2313.    0. 1163. 2315.
      0.]
  [   0.    0. 1718.    0.    0. 1745.    0.    0. 1243.    0. 1160.
      0.]]]

Cumulative Demand[2]:
[[[    0.  3773.     0.     0.  2257.     0.  1091.  2179.     0.  1097.
    2169.     0.]
  [    0.     0.  2680.     0.     0.     0.     0.  2000.     0.     0.
       0.   445.]
  [ 1010. 11637.     0.     0.     0.     0.     0.     

In [16]:
def update_inventory(I, cumulative_Demand_list, Demand):
    updated_inventories = []
    
    for cumu_Demand in cumulative_Demand_list:
        I_copy = np.zeros_like(Demand)  # Create a new I_copy for each cumulative_Demand
        for i in range(I_copy.shape[0]):
            for j in range(I_copy.shape[1]):
                # Initialize the first period inventory
                I_copy[i, j, 0] = I[i, j, 0] + cumu_Demand[i, j, 0] - Demand[i, j, 0]
                # Update inventory for subsequent periods
                for k in range(1, I.shape[2]):
                    I_copy[i, j, k] = I_copy[i, j, k-1] + cumu_Demand[i, j, k] - Demand[i, j, k]
        updated_inventories.append(I_copy)
    return updated_inventories
updated_inventories = update_inventory(I, cumulative_Demand_list, Demand)
for idx, inventory in enumerate(updated_inventories, start=1):
    print(f"Updated Inventory Levels for Chromosome {idx}:\n{inventory}\n")


Updated Inventory Levels for Chromosome 1:
[[[2.960e+02 2.960e+02 2.960e+02 2.960e+02 2.960e+02 2.479e+03 1.388e+03
   2.960e+02 1.393e+03 2.960e+02 1.405e+03 2.960e+02]
  [5.650e+02 1.610e+02 1.610e+02 1.610e+02 3.389e+03 2.796e+03 2.161e+03
   1.647e+03 1.170e+03 6.610e+02 1.610e+02 1.610e+02]
  [6.000e+02 6.000e+02 1.676e+03 6.000e+02 2.701e+03 1.688e+03 6.000e+02
   3.767e+03 2.657e+03 1.688e+03 6.000e+02 6.000e+02]]

 [[7.070e+02 7.070e+02 3.644e+03 2.615e+03 1.611e+03 7.070e+02 7.070e+02
   1.734e+03 7.070e+02 7.070e+02 7.070e+02 7.070e+02]
  [2.180e+02 2.180e+02 2.180e+02 2.180e+02 1.574e+03 8.600e+02 2.180e+02
   9.780e+02 2.180e+02 1.587e+03 7.590e+02 2.180e+02]
  [9.920e+02 4.980e+02 0.000e+00 0.000e+00 0.000e+00 4.830e+02 0.000e+00
   0.000e+00 0.000e+00 5.320e+02 0.000e+00 0.000e+00]]

 [[2.000e+02 2.000e+02 4.490e+02 2.000e+02 1.173e+03 8.360e+02 4.930e+02
   2.000e+02 2.000e+02 5.270e+02 2.000e+02 2.000e+02]
  [4.049e+03 2.926e+03 1.754e+03 5.000e+02 5.000e+02 5.000e+02 5

In [17]:
def fitness(modified_v_list, cumulative_Demand_list, Demand, hc, oc, pc, updated_inventories):
    m = len(modified_v_list)
    Totalcost = np.zeros(m)
    Ordering = np.zeros((m, Demand.shape[0], Demand.shape[1]))
    Purchasing = np.zeros((m, Demand.shape[0], Demand.shape[1]))
    Holding = np.zeros((m, Demand.shape[0], Demand.shape[1]))

    for idx in range(m):
        modified_v = modified_v_list[idx]
        cumulative_Demand = cumulative_Demand_list[idx]
        Inv = updated_inventories[idx]

        for i in range(modified_v.shape[0]):
            for j in range(modified_v.shape[1]):
                Ordering[idx, i, j] = oc * np.sum(modified_v[i, j, :])
                Purchasing[idx, i, j] = pc[j] * np.sum(cumulative_Demand[i, j, :])
                Holding[idx, i, j] = hc[j] * np.sum(Inv[i, j, :])  

        Totalcost[idx] = np.sum(Ordering[idx, :, :] + Purchasing[idx, :, :] + Holding[idx, :, :])
    return Totalcost
costs = fitness(modified_v_list, cumulative_Demand_list, Demand, hc, oc, pc, updated_inventories)
for idx, cost in enumerate(costs):
    print(f"Total cost for v[{idx}]: {cost}")

Total cost for v[0]: 2946394900.0
Total cost for v[1]: 2962923600.0
Total cost for v[2]: 2951975000.0
Total cost for v[3]: 2948792000.0
Total cost for v[4]: 2953696300.0
Total cost for v[5]: 2946116400.0
Total cost for v[6]: 2950711100.0
Total cost for v[7]: 2956478800.0
Total cost for v[8]: 2947876500.0
Total cost for v[9]: 2945036100.0
Total cost for v[10]: 2945870300.0
Total cost for v[11]: 2949543300.0
Total cost for v[12]: 2949172800.0
Total cost for v[13]: 2950343600.0
Total cost for v[14]: 2943277800.0
Total cost for v[15]: 2950264300.0
Total cost for v[16]: 2946211200.0
Total cost for v[17]: 2952517000.0
Total cost for v[18]: 2957174000.0
Total cost for v[19]: 2948291800.0
Total cost for v[20]: 2946676500.0
Total cost for v[21]: 2952690600.0
Total cost for v[22]: 2949284300.0
Total cost for v[23]: 2949133600.0
Total cost for v[24]: 2949723400.0
Total cost for v[25]: 2949014900.0
Total cost for v[26]: 2949009300.0
Total cost for v[27]: 2956347200.0
Total cost for v[28]: 29458158

In [18]:
def dvar(modified_v_list, cumulative_Demand_list, Demand, hc, oc, pc, updated_inventories):
    m = len(modified_v_list)
    Ordering = np.zeros((m, Demand.shape[0], Demand.shape[1]))
    Purchasing = np.zeros((m, Demand.shape[0], Demand.shape[1]))
    Holding = np.zeros((m, Demand.shape[0], Demand.shape[1]))

    for idx in range(m):
        modified_v = modified_v_list[idx]
        cumulative_Demand = cumulative_Demand_list[idx]
        Inv = updated_inventories[idx]

        for i in range(modified_v.shape[0]):
            for j in range(modified_v.shape[1]):
                Ordering[idx, i, j] = oc * np.sum(modified_v[i, j, :])
                Purchasing[idx, i, j] = pc[j] * np.sum(cumulative_Demand[i, j, :])
                Holding[idx, i, j] = hc[j] * np.sum(Inv[i, j, :])  

    return Ordering, Purchasing, Holding

ordering, purchasing, holding = dvar(modified_v_list, cumulative_Demand_list, Demand, hc, oc, pc, updated_inventories)
for idx in range(len(ordering)):
    print(f"Chromosome {idx + 1}:")
    print("Ordering Cost:")
    print(ordering[idx])
    print("Purchasing Cost:")
    print(purchasing[idx])
    print("Holding Cost:")
    print(holding[idx])
    print()


Chromosome 1:
Ordering Cost:
[[3500000. 2000000. 3000000.]
 [3500000. 3000000. 4000000.]
 [3500000. 3500000. 2000000.]]
Purchasing Cost:
[[6.2830e+08 1.0250e+08 5.0588e+08]
 [5.5890e+08 1.5134e+08 2.4784e+08]
 [1.7540e+08 2.8156e+08 2.3464e+08]]
Holding Cost:
[[4516500. 2638800. 7110800.]
 [7630000. 1456800. 1002000.]
 [2439000. 3100200. 2140800.]]

Chromosome 2:
Ordering Cost:
[[3000000. 1500000. 1000000.]
 [3500000. 3000000. 4500000.]
 [2000000. 3000000. 3000000.]]
Purchasing Cost:
[[6.2830e+08 1.0250e+08 5.0588e+08]
 [5.5890e+08 1.5134e+08 2.4784e+08]
 [1.7540e+08 2.8156e+08 2.3464e+08]]
Holding Cost:
[[ 5435000.  2237000. 25972000.]
 [ 6828000.  1518200.   781200.]
 [ 4838000.  3021400.  1432800.]]

Chromosome 3:
Ordering Cost:
[[2000000. 2000000. 4000000.]
 [3000000. 3000000. 5000000.]
 [4000000. 3000000. 2500000.]]
Purchasing Cost:
[[6.2830e+08 1.0250e+08 5.0588e+08]
 [5.5890e+08 1.5134e+08 2.4784e+08]
 [1.7540e+08 2.8156e+08 2.3464e+08]]
Holding Cost:
[[11143500.  1359600.  5030

In [19]:
def rank_fitness(costs):
    sorted_indices = np.argsort(costs)[::1]  # Sort indices in descending order based on costs
    sorted_costs = costs[sorted_indices]    # Sort costs accordingly
    return sorted_indices
sorted_indices = rank_fitness(costs)
sorted_costs = costs[sorted_indices]

for idx, cost in zip(sorted_indices, sorted_costs):
    print(f"Total cost for v[{idx}]: {cost}")

Total cost for v[37]: 2942984900.0
Total cost for v[14]: 2943277800.0
Total cost for v[78]: 2943321600.0
Total cost for v[31]: 2943370300.0
Total cost for v[63]: 2943439100.0
Total cost for v[9]: 2945036100.0
Total cost for v[85]: 2945046600.0
Total cost for v[86]: 2945146700.0
Total cost for v[36]: 2945173500.0
Total cost for v[98]: 2945373300.0
Total cost for v[74]: 2945435800.0
Total cost for v[38]: 2945717900.0
Total cost for v[28]: 2945815800.0
Total cost for v[71]: 2945850800.0
Total cost for v[10]: 2945870300.0
Total cost for v[91]: 2946003100.0
Total cost for v[32]: 2946065600.0
Total cost for v[46]: 2946094200.0
Total cost for v[5]: 2946116400.0
Total cost for v[16]: 2946211200.0
Total cost for v[45]: 2946239500.0
Total cost for v[56]: 2946306700.0
Total cost for v[90]: 2946369800.0
Total cost for v[0]: 2946394900.0
Total cost for v[50]: 2946594200.0
Total cost for v[20]: 2946676500.0
Total cost for v[35]: 2946856600.0
Total cost for v[62]: 2946920300.0
Total cost for v[89]: 2

In [20]:
def rank_pop(costs):
    sorted_indices = np.argsort(costs)[::1]  # Sort indices in descending order based on costs
    sorted_costs = costs[sorted_indices]    # Sort costs accordingly
    return sorted_indices
ranked_indices = rank_pop(costs)
for idx in ranked_indices:
    print(f"v[{idx}]")

v[37]
v[14]
v[78]
v[31]
v[63]
v[9]
v[85]
v[86]
v[36]
v[98]
v[74]
v[38]
v[28]
v[71]
v[10]
v[91]
v[32]
v[46]
v[5]
v[16]
v[45]
v[56]
v[90]
v[0]
v[50]
v[20]
v[35]
v[62]
v[89]
v[58]
v[72]
v[77]
v[8]
v[82]
v[80]
v[53]
v[93]
v[19]
v[51]
v[97]
v[3]
v[59]
v[76]
v[26]
v[25]
v[23]
v[12]
v[94]
v[22]
v[43]
v[83]
v[11]
v[24]
v[67]
v[33]
v[95]
v[57]
v[68]
v[48]
v[15]
v[13]
v[69]
v[96]
v[40]
v[6]
v[41]
v[75]
v[39]
v[65]
v[2]
v[54]
v[17]
v[66]
v[21]
v[79]
v[42]
v[34]
v[52]
v[64]
v[73]
v[99]
v[30]
v[4]
v[81]
v[44]
v[87]
v[70]
v[84]
v[55]
v[60]
v[27]
v[7]
v[18]
v[88]
v[49]
v[1]
v[47]
v[92]
v[29]
v[61]


In [21]:
def ga_generate(modified_v_list, ranked_indices, m, shape, pselect, pmutation):
    n, o, p = shape  # Dimensions of each chromosome
    popu = np.zeros((m, n, o, p), dtype=int)  # New population
    ms = round(m * pselect)  # Number of selected parents
    mm = ms + round((m - ms) * pmutation)  # Number of mutated individuals
    parent = np.zeros((ms, n, o, p), dtype=int)  # Array to store selected parents
    
    # Copy values from modified_v_list to parent and popu arrays
    for i in range(ms):
        parent[i] = modified_v_list[ranked_indices[i]]  # Assign ranked individuals to parent
        popu[i] = parent[i]  # Copy to popu as well
    
    # Mutating some individuals
    for i in range(ms, mm):
        j = i - ms
        if j < ms:
            popu[i] = parent[j]
        else:
            j = np.random.randint(0, ms, dtype=int)
            popu[i] = parent[j]
        
        # Randomly select mutation points within the chromosome
        t = np.random.randint(1, p, dtype=int)
        l = random.choice(list(range(1, t)) + list(range(t + 1, p)))
        
        # Mutation
        x, y = np.random.randint(0, n, dtype=int), np.random.randint(0, o, dtype=int)
        if popu[i][x][y][t] == 0:
            popu[i][x][y][t] = 1
        else:
            popu[i][x][y][t] = 0
        
        x, y = np.random.randint(0, n, dtype=int), np.random.randint(0, o, dtype=int)
        if popu[i][x][y][l] == 0:
            popu[i][x][y][l] = 1
        else:
            popu[i][x][y][l] = 0
    
    # Crossover
    for i in range(mm, m-1, 2):
        par1 = np.random.randint(0, ms, dtype=int)
        popu[i] = parent[par1]
        
        par2 = random.choice(list(range(0, par1)) + list(range(par1 + 1, ms)))
        popu[i + 1] = parent[par2]
        
        # Randomly select crossover point within the chromosome
        p_cross = np.random.randint(1, p, dtype=int)
        x_cross, y_cross = np.random.randint(0, n, dtype=int), np.random.randint(0, o, dtype=int)
        for a in range(p_cross):
            popu[i][x_cross][y_cross][a] = parent[par2][x_cross][y_cross][a]
            popu[i + 1][x_cross][y_cross][a] = parent[par1][x_cross][y_cross][a]
    
    return popu
popp = ga_generate(modified_v_list, ranked_indices, m, shape, pselect, pmutation)
print(popp)

[[[[0 1 1 1 1 0 1 1 1 1 0 1]
   [0 0 1 0 0 1 1 1 1 0 0 1]
   [1 0 1 1 1 1 1 1 1 0 1 1]]

  [[0 1 0 0 1 0 1 1 0 1 1 1]
   [0 1 0 1 0 0 0 0 1 0 0 0]
   [1 1 1 0 0 1 1 1 1 1 0 0]]

  [[1 1 1 0 0 0 1 0 0 1 0 1]
   [1 1 0 0 1 0 0 1 0 1 1 1]
   [0 0 1 1 1 1 0 0 1 0 1 1]]]


 [[[0 1 1 1 0 1 1 1 1 1 0 1]
   [0 0 1 0 0 0 1 0 0 1 1 0]
   [1 0 0 1 0 1 0 1 0 0 1 0]]

  [[0 1 1 0 1 0 0 1 1 1 1 0]
   [0 1 0 1 1 1 0 0 1 1 0 0]
   [1 0 1 1 0 1 0 0 0 1 0 1]]

  [[1 1 0 0 1 1 1 1 1 0 1 1]
   [1 0 0 0 1 0 1 1 1 1 1 0]
   [0 0 1 1 1 0 1 1 0 1 0 0]]]


 [[[0 1 1 1 1 1 1 1 1 1 0 0]
   [0 0 1 1 1 1 0 1 1 1 1 1]
   [1 1 0 1 1 1 0 1 0 1 1 0]]

  [[0 1 1 0 1 0 1 1 0 1 1 0]
   [0 1 1 1 0 1 0 0 1 0 1 0]
   [1 1 1 0 1 0 1 0 0 0 1 1]]

  [[1 0 1 0 0 1 1 1 0 1 0 1]
   [1 1 0 1 0 0 1 1 0 1 1 0]
   [0 0 1 1 0 0 1 0 1 0 0 0]]]


 [[[0 1 0 1 1 1 0 1 1 0 1 0]
   [0 0 1 1 0 1 1 0 0 0 0 0]
   [1 1 1 0 1 0 1 1 1 0 1 0]]

  [[0 1 0 1 0 0 1 1 0 0 1 1]
   [0 1 0 0 1 0 0 1 1 0 1 1]
   [1 1 0 0 1 0 0 1 1 1 1 0]]

  [[1 0 0 1 1 0

In [22]:
import time

print("GENETIC ALGORITHM OPTIMIZATION")

# Initialization
start_time = time.time()
iter = 0
v_list = [initialize(*shape) for _ in range(m)]
modified_v_list = check_chromosome(v_list, Demand, WU, initial_inventory)
cumulative_Demand_list = cumuD(modified_v_list, Demand)
updated_inventories = update_inventory(I, cumulative_Demand_list, Demand)
costs = fitness(modified_v_list, cumulative_Demand_list, Demand, hc, oc, pc, updated_inventories)
ordering, purchasing, holding = dvar(modified_v_list, cumulative_Demand_list, Demand, hc, oc, pc, updated_inventories)
sorted_indices = rank_fitness(costs)
ranked_indices = rank_pop(costs)
popp = ga_generate(modified_v_list, ranked_indices, m, shape, pselect, pmutation)

# Track best solution information
best_solution_index = ranked_indices[0]  # Best solution index
best_modified_v_list = modified_v_list[best_solution_index]
best_cumulative_Demand_list = cumulative_Demand_list[best_solution_index]
best_updated_inventories = updated_inventories[best_solution_index]
best_ordering = ordering[best_solution_index]
best_purchasing = purchasing[best_solution_index]
best_holding = holding[best_solution_index]
best_cost = costs[best_solution_index]

while iter < maxI_ga:
    iter += 1
    modified_v_list = check_chromosome(v_list, Demand, WU, initial_inventory)
    cumulative_Demand_list = cumuD(modified_v_list, Demand)
    updated_inventories = update_inventory(I, cumulative_Demand_list, Demand)
    costs = fitness(modified_v_list, cumulative_Demand_list, Demand, hc, oc, pc, updated_inventories)
    ordering, purchasing, holding = dvar(modified_v_list, cumulative_Demand_list, Demand, hc, oc, pc, updated_inventories)
    sorted_indices = rank_fitness(costs)
    ranked_indices = rank_pop(costs)
    popp = ga_generate(modified_v_list, ranked_indices, m, shape, pselect, pmutation)
    
    # Update best solution information if a better solution is found
    if costs[ranked_indices[0]] < best_cost:
        best_solution_index = ranked_indices[0]
        best_modified_v_list = modified_v_list[best_solution_index]
        best_cumulative_Demand_list = cumulative_Demand_list[best_solution_index]
        best_updated_inventories = updated_inventories[best_solution_index]
        best_ordering = ordering[best_solution_index]
        best_purchasing = purchasing[best_solution_index]
        best_holding = holding[best_solution_index]
        best_cost = costs[best_solution_index]

# Print the best solution and time taken
print(f"Best solution found:")
print(f"Modified chromosomes (v_list): {best_modified_v_list}")
print(f"Cumulative demand list: {best_cumulative_Demand_list}")
print(f"Updated inventories: {best_updated_inventories}")
print(f"Decision variables - Ordering: {best_ordering}, Purchasing: {best_purchasing}, Holding: {best_holding}")
print(f"Total cost of best solution: {best_cost}")

# Calculate and print time taken
end_time = time.time()
print(f"Time taken to run the code: {end_time - start_time} seconds")
print(f"Time taken to run the code: {(end_time - start_time) / 3600} hours")


GENETIC ALGORITHM OPTIMIZATION
Best solution found:
Modified chromosomes (v_list): [[[0 1 1 1 0 1 1 0 1 1 1 1]
  [0 0 1 0 1 0 0 0 0 1 0 1]
  [1 0 1 0 1 1 1 0 1 1 1 1]]

 [[0 1 1 1 0 1 0 0 1 1 0 1]
  [0 1 0 0 0 0 1 0 1 0 0 1]
  [1 1 0 1 1 1 1 0 1 0 0 1]]

 [[1 1 0 1 0 1 1 0 0 1 1 1]
  [1 0 0 1 0 1 0 1 0 1 1 1]
  [0 0 1 1 1 0 1 1 0 1 0 1]]]
Cumulative demand list: [[[   0. 1144. 1301. 2420.    0. 1165. 2183.    0. 1087. 1097. 1060.
   1109.]
  [   0.    0.  899.    0. 2772.    0.    0.    0.    0. 1009.    0.
    445.]
  [2140.    0. 2155.    0.  981. 1013. 2158.    0. 1110.  969. 1088.
   1033.]]

 [[   0. 1204. 1021. 2033.    0. 2950.    0.    0. 1027. 1977.    0.
    966.]
  [   0. 3603.    0.    0.    0.    0. 1273.    0. 2150.    0.    0.
    541.]
  [ 545.  992.    0.  520.  501.  448. 1018.    0. 1633.    0.    0.
    539.]]

 [[ 243.  613.    0.  484.    0.  337.  890.    0.    0.  348.  327.
    266.]
  [3558.    0.    0. 2481.    0. 2248.    0. 2313.    0. 1163. 1192.
   1123.]